# Install requirements

In [ ]:
!pip3 install openmim
!mim install mmengine

In [ ]:
!mim install "mmcv>=2.0.0"

In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .

In [ ]:
!sed -i -e 's/2.2.0/2.2.1/g' ./mmdet/__init__.py

In [ ]:
import mmdet
print(mmdet.__version__)

# Dataset preparation

In [ ]:
!gdown 1JGYVpDE1jb3sIMJd2Cxw0-p_FoYMDs-r
!unzip -q augmented_data_v1.zip

In [ ]:
!pip install -q globox
from globox import AnnotationSet
yolo_train_set = AnnotationSet.from_yolo_v5(
    folder="./augmented_data_v1/train/labels/",
    image_folder="./augmented_data_v1/train/images/"
)
yolo_val_set = AnnotationSet.from_yolo_v5(
    folder="./augmented_data_v1/valid/labels/",
    image_folder="./augmented_data_v1/valid/images/"
)
yolo_train_set.save_coco("./augmented_data_v1/train.json", auto_ids=True)
yolo_val_set.save_coco("./augmented_data_v1/valid.json", auto_ids=True)

# TOOD

In [ ]:
%cd mmdetection

In [ ]:
!mim download mmdet --config tood_r50_fpn_1x_coco --dest .

processing tood_r50_fpn_1x_coco...
downloading ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 MiB 28.4 MB/s eta 0:00:00
Successfully downloaded tood_r50_fpn_1x_coco_20211210_103425-20e20746.pth to /content/mmdetection
Successfully dumped tood_r50_fpn_1x_coco.py to /content/mmdetection


In [ ]:
%%writefile ./configs/tood/tood_r50_fpn_1x_sino_nom.py
# Inherit and overwrite part of the config based on this config
_base_ = './tood_r50_fpn_1x_coco.py'

data_root = './augmented_data_v1/' # dataset root

train_batch_size_per_gpu = 2
train_num_workers = 2

max_epochs = 20
stage2_num_epochs = 1
base_lr = 0.00008


metainfo = {
    'classes': ('0', ),
    'palette': [
        (220, 20, 60),
    ]
}

train_dataloader = dict(
    batch_size=train_batch_size_per_gpu,
    num_workers=train_num_workers,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='train/images/'),
        ann_file='train.json'))

val_dataloader = dict(
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='valid/images/'),
        ann_file='valid.json'))

test_dataloader = val_dataloader

val_evaluator = dict(ann_file=data_root + 'valid.json')

test_evaluator = val_evaluator

model = dict(bbox_head=dict(num_classes=1), test_cfg=dict(score_thr=0.5, max_per_img=400))

# learning rate
# param_scheduler = [
#     dict(
#         type='LinearLR',
#         start_factor=1.0e-5,
#         by_epoch=False,
#         begin=0,
#         end=10),
#     dict(
#         # use cosine lr from 10 to 20 epoch
#         type='CosineAnnealingLR',
#         eta_min=base_lr * 0.05,
#         begin=max_epochs // 2,
#         end=max_epochs,
#         T_max=max_epochs // 2,
#         by_epoch=True,
#         convert_to_iter_based=True),
# ]

train_pipeline_stage2 = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='RandomResize',
        scale=(640, 640),
        ratio_range=(0.1, 2.0),
        keep_ratio=True),
    dict(type='RandomCrop', crop_size=(640, 640)),
    dict(type='YOLOXHSVRandomAug'),
    dict(type='RandomFlip', prob=0.5),
    dict(type='Pad', size=(640, 640), pad_val=dict(img=(114, 114, 114))),
    dict(type='PackDetInputs')
]

# optimizer
# optim_wrapper = dict(
#     _delete_=True,
#     type='OptimWrapper',
#     optimizer=dict(type='AdamW', lr=base_lr, weight_decay=0.05),
#     paramwise_cfg=dict(
#         norm_decay_mult=0, bias_decay_mult=0, bypass_duplicate=True))

default_hooks = dict(
    checkpoint=dict(
        interval=5,
        max_keep_ckpts=2,  # only keep latest 2 checkpoints
        save_best='auto'
    ),
    logger=dict(type='LoggerHook', interval=20))

custom_hooks = [
    dict(
        type='PipelineSwitchHook',
        switch_epoch=max_epochs - stage2_num_epochs,
        switch_pipeline=train_pipeline_stage2)
]

# load COCO pre-trained weight
load_from = './tood_r50_fpn_1x_coco_20211210_103425-20e20746.pth'

train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=max_epochs, val_interval=1)
visualizer = dict(vis_backends=[dict(type='LocalVisBackend'),dict(type='TensorboardVisBackend')])

Writing ./configs/tood/tood_r50_fpn_1x_sino_nom.py


In [ ]:
!python tools/train.py configs/tood/tood_r50_fpn_1x_sino_nom.py

05/22 09:06:08 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1829151309
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.2, V12.2.140
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.3.0+cu121
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.6 (Git Hash 86e6af5974177e513fd3fee58425e1063e7f1361)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.2
  - Built with CUDA Runtime 12.1
  - NVCC architecture flag

In [ ]:
!cd work_dirs; zip -r tood_r50_fpn_1x_sino_nom.zip ./tood_r50_fpn_1x_sino_nom

  adding: tood_r50_fpn_1x_sino_nom/ (stored 0%)
  adding: tood_r50_fpn_1x_sino_nom/epoch_15.pth (deflated 7%)
  adding: tood_r50_fpn_1x_sino_nom/epoch_20.pth (deflated 7%)
  adding: tood_r50_fpn_1x_sino_nom/best_coco_bbox_mAP_epoch_9.pth (deflated 7%)
  adding: tood_r50_fpn_1x_sino_nom/tood_r50_fpn_1x_sino_nom.py (deflated 77%)
  adding: tood_r50_fpn_1x_sino_nom/last_checkpoint (deflated 7%)
  adding: tood_r50_fpn_1x_sino_nom/20240522_090605/ (stored 0%)
  adding: tood_r50_fpn_1x_sino_nom/20240522_090605/20240522_090605.log (deflated 86%)
  adding: tood_r50_fpn_1x_sino_nom/20240522_090605/vis_data/ (stored 0%)
  adding: tood_r50_fpn_1x_sino_nom/20240522_090605/vis_data/events.out.tfevents.1716368772.a01eeda05e8c.5417.0 (deflated 68%)
  adding: tood_r50_fpn_1x_sino_nom/20240522_090605/vis_data/config.py (deflated 77%)
  adding: tood_r50_fpn_1x_sino_nom/20240522_090605/vis_data/20240522_090605.json (deflated 75%)
  adding: tood_r50_fpn_1x_sino_nom/20240522_090605/vis_data/scalars.json (d

In [ ]:
!cd work_dirs; curl -F "file=@tood_r50_fpn_1x_sino_nom.zip" https://temp.sh/upload

<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.18.0 (Ubuntu)</center>
</body>
</html>


In [ ]:
!cd work_dirs; curl -F "file=@tood_r50_fpn_1x_sino_nom.zip" https://file.io

{"success":true,"status":200,"id":"ae484020-1822-11ef-9dcc-f7f0d5a323d1","key":"DdwQ7SwvBb1W","path":"/","nodeType":"file","name":"tood_r50_fpn_1x_sino_nom.zip","title":null,"description":null,"size":596682504,"link":"https://file.io/DdwQ7SwvBb1W","private":false,"expires":"2024-06-05T10:04:30.315Z","downloads":0,"maxDownloads":1,"autoDelete":true,"planId":0,"screeningStatus":"pending","mimeType":"application/octet-stream","created":"2024-05-22T10:04:30.315Z","modified":"2024-05-22T10:04:30.315Z"}